**Mask-RCNN setup**

In [2]:
# uninstall improper package versions
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

In [6]:
# reinstall with proper versions
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install opencv-python
!pip install scikit-image==0.17.2
!pip install mask-rcnn==2.1

You should consider upgrading via the 'C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached scikit_image-0.17.2-cp37-cp37m-win_amd64.whl (11.5 MB)

You should consider upgrading via the 'C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached tifffile-2021.11.2-py3-none-any.whl (178 kB)
  Using cached Pillow-8.4.0-cp37-cp37m-win_amd64.whl (3.2 MB)
  Using cached kiwisolver-1.3.2-cp37-cp37m-win_amd64.whl (51 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


In [14]:
#might not be needed
# import mask rcnn and set up
#%cd
#!git clone https://github.com/matterport/Mask_RCNN.git
#%cd Mask_RCNN/
#!python setup.py install
#!pip show mask-rcnn

C:\Users\a_sta
C:\Users\a_sta\Mask_RCNN


fatal: destination path '.' already exists and is not an empty directory.


running install
running bdist_egg

C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\lib\site-packages\setuptools\dist.py:694: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\lib\site-packages\setuptools\dist.py:694: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead



running egg_info
writing mask_rcnn.egg-info\PKG-INFO
writing dependency_links to mask_rcnn.egg-info\dependency_links.txt
writing top-level names to mask_rcnn.egg-info\top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mask_rcnn.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\config.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\model.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\parallel_model.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\utils.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\visualize.py -> build\bdist.win-amd64\egg\mrcnn
copying build\lib\mrcnn\__init__.py -> build\bdist.win-amd64\egg\mrcnn
byte-compiling build\bdist.win-amd64\egg\mrcnn\config.py to config.cpython-37.pyc
byte-compiling build

  % (opt, underscore_opt))
C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\venv\lib\site-packages\setuptools\dist.py:694: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
zip_safe flag not set; analyzing archive contents...


Name: mask-rcnn
Version: 2.1
Summary: Mask R-CNN for object detection and instance segmentation
Home-page: https://github.com/matterport/Mask_RCNN
Author: Matterport
Author-email: waleed.abdulla@gmail.com
License: MIT
Location: c:\users\a_sta\onedrive - worcester polytechnic institute (wpi.edu)\2021-22\cs 539\final project\venv\lib\site-packages\mask_rcnn-2.1-py3.7.egg
Requires: 
Required-by: 


In [20]:
import json
import os
import sys
import numpy as np
import cv2
from mrcnn import utils
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import skimage.draw
import urllib.request



######################################
# Configuration
######################################
class CustomConfig(Config):
    NAME = "custom_mcrnn"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3 # ?? classes + background
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
    LEARNING_RATE = .001
    BATCH_SIZE = 10

config = CustomConfig()


#######################################
# Dataset
#######################################
class CustomDataset(utils.Dataset):

    def load_dataset(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        #add all new classes
        for i, c in enumerate(CLASSES):
            self.add_class("object", i, c)

        # Train or validation dataset?
        dataset_dir = DATASET_DIR
        annotations = json.load(open(BASE_ANNOTATIONS_FILE))
       
        #go through each image
        for a in annotations["imgs"].values():

            polygons = []
            for instance in a["objects"]:
                if "polygon" in instance.keys():
                    polygons.append(instance["polygon"])

            image_path = os.path.join(dataset_dir, a['path'])
            if os.path.exists(image_path):
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]
                self.add_image(
                    "sign",
                    image_id=a["id"],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)


    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "balloon":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            all_x, all_y = zip(*p) #unzip the list
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(all_y, all_x)
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)


#######################################
# Training
#######################################
print(os.path.abspath(os.getcwd()))
# define constants
DATASET_DIR = 'data' # directory where train images can be found
BASE_ANNOTATIONS_FILE = 'data/annotations.json' # file with annotations

TRAIN_TEST_SPLIT = .8 # proportion of images to use for training set, remainder will be reserved for validation
CLASSES = [] # all annotation classes, to generate from the json

# set up train and validation data

dataset_train = CustomDataset()
dataset_train.load_dataset(DATASET_DIR, "train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_dataset(DATASET_DIR, "test")
dataset_val.prepare()


if not os.path.exists("mask_rcnn_coco.h5"):
    urllib.request.urlretrieve("https://github.com/matterport/Mask_RCNN/releases/download/v1.0/mask_rcnn_coco.h5", "mask_rcnn_coco.h5")
model = MaskRCNN(mode='training', model_dir='./model/', config=CustomConfig())

model.load_weights("mask_rcnn_coco.h5", by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# print summary
print(model.keras_model.summary())

# train model
model.train(train_dataset=dataset_train,
            val_dataset=dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads')

# save training results to external file
model.keras_model.save_weights('model.h5')

C:\Users\a_sta\OneDrive - Worcester Polytechnic Institute (wpi.edu)\2021-22\CS 539\Final Project\source
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_image (InputLayer)         (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D) (None, None, None, 3) 0           input_image[0][0]                
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_2[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNorm)             (None, None, None, 64 256         conv1[0][0]          

c:\users\a_sta\onedrive - worcester polytechnic institute (wpi.edu)\2021-22\cs 539\final project\venv\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


KeyboardInterrupt: 